# Hyperparameter Tuning

## Setup

In [ ]:
# Import packages 
import numpy as np

from tqdm import tqdm, trange 
import glob
import time

from PIL import Image
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import DataLoader, random_split, Dataset

import torchvision
import torchvision.transforms.functional as TF

import albumentations as A
from albumentations.pytorch import ToTensorV2



## Model 

In [ ]:
# double convolutional layer which is executed in every step of the u-net 
# conv layer takes as input number of input channels -> in_channels and outputs vice versa
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    # forward pass in the conv layer 
    def forward(self, x):
        return self.conv(x)

# design complete u-net shape 
# model takes as default 3 input channels and 6 output channels
class UNET(nn.Module):
    def __init__(
            self, in_channels=3, out_channels=6, features=[64, 128, 256, 512],  # features -> num of input nodes at every stage in the model 
    ):
        super(UNET, self).__init__()
        self.downs = nn.ModuleList()
        self.ups = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Down part of UNET
        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature

        # Up part of UNET
        for feature in reversed(features):  # reverse the features i.o. to move upwards in the model 
            self.ups.append(
                nn.ConvTranspose2d(
                    feature*2, feature, kernel_size=2, stride=2,
                )
            )
            self.ups.append(DoubleConv(feature*2, feature))
        
        # lowest stage in u-net 
        self.bottleneck = DoubleConv(features[-1], features[-1]*2)
        # final conv layer: takes in 64 channels and outputs 1 channel by default 
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    # forward pass of the u-net model between stages 
    def forward(self, x):
        skip_connections = []  # red arrows in the model representation 

        for down in self.downs:
            x = down(x)  # one DoubleConv run-through 
            skip_connections.append(x)
            x = self.pool(x)

        x = self.bottleneck(x)
        skip_connections = skip_connections[::-1]

        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x)
            skip_connection = skip_connections[idx//2]

            if x.shape != skip_connection.shape:
                x = TF.resize(x, size=skip_connection.shape[2:])

            concat_skip = torch.cat((skip_connection, x), dim=1)
            x = self.ups[idx+1](concat_skip)

        return self.final_conv(x)

## Import Data

In [ ]:
# Establish connection with google drive
from google.colab import drive
drive.mount('/content/drive')

# Set path variable
path = "/content/drive/MyDrive/DS-Project/"

In [ ]:
# Create lists with paths to all image and label data 
imgs = glob.glob(path+'data/model_training/2_Ortho_RGB/sliced/*tif')
labels = glob.glob(path+"data/model_training/Labels_all/sliced/*tif")

# Create dictionary -> {key: 'link/to/image_or_label'}
labels_dict = {label.split("/")[-1].split(".")[0].rsplit('_', 1)[0] : label for label in labels}
imgs_dict = {img.split("/")[-1].split(".")[0].rsplit('_', 1)[0] : img for img in imgs}

# Create list with all keys 
keys = sorted(list(set(imgs_dict)))

## Custom Dataset 

In [ ]:
class PotsdamDataset(Dataset):
    def __init__(self, imgs_dict, labels_dict, keys, transform=None):
        self.img_dir = imgs_dict
        self.mask_dir = labels_dict
        self.keys = keys
        self.transform = transform
        
        self.RGB_classes = {
            'imprevious' : [255, 255, 225],
            'building' : [0,  0, 255],
            'low_vegetation' : [0, 255, 255],
            'tree' : [0,  255,  0], 
            'car' : [ 255, 255, 0],
            'background' : [255, 0, 0]
            }  # in RGB
        
        self.bin_classes = ['imprevious', 'building', 'low_vegetation', 'tree', 'car', 'background']

    def __len__(self):
        return len(self.keys)

    def __getitem__(self, idx):
        img_path = self.img_dir[self.keys[idx]]
        mask_path = self.mask_dir[self.keys[idx]]
        
        image = np.array(Image.open(img_path).convert("RGB"))
        mask = np.array(Image.open(mask_path).convert("RGB"))
        
        cls_mask = np.zeros(mask.shape) # dim: (6000, 6000, 3)
        
        cls_mask[(mask == self.RGB_classes['imprevious']).all(-1)] = self.bin_classes.index('imprevious')
        cls_mask[(mask == self.RGB_classes['building']).all(-1)] = self.bin_classes.index('building')
        cls_mask[(mask == self.RGB_classes['low_vegetation']).all(-1)] = self.bin_classes.index('low_vegetation')
        cls_mask[(mask == self.RGB_classes['tree']).all(-1)] = self.bin_classes.index('tree')
        cls_mask[(mask == self.RGB_classes['car']).all(-1)] = self.bin_classes.index('car')
        cls_mask[(mask == self.RGB_classes['background']).all(-1)] = self.bin_classes.index('background')
        cls_mask = cls_mask[:,:,0] # omit last dimension (, , 3) -> RGB  

        if self.transform is not None:
            augmentations = self.transform(image=image, mask=cls_mask)
            image = augmentations["image"]
            mask = augmentations["mask"]

        return image, mask

## Util Functions

### Train-/Validation-Split 

In [ ]:
dataset = PotsdamDataset(imgs_dict, labels_dict, keys)
validation_split = .2
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
    
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

### Data Loader

In [ ]:
def get_loaders(
    imgs_dict,
    labels_dict,
    keys,
    batch_size,
    train_transform,
    val_transform,
    num_workers = 2,
    pin_memory = True,
):
    
    train_data = PotsdamDataset(
        imgs_dict = imgs_dict,
        labels_dict = labels_dict,
        keys = keys, 
        transform = train_transform,
    )
    
    valid_data = PotsdamDataset(
        imgs_dict = imgs_dict,
        labels_dict = labels_dict,
        keys = keys, 
        transform = val_transform,
    )

    train_loader = DataLoader(
        train_data,
        batch_size = batch_size,
        num_workers = num_workers,
        pin_memory = pin_memory,
        sampler = train_sampler,
    )

    val_loader = DataLoader(
        valid_data,
        batch_size = batch_size,
        num_workers = num_workers,
        pin_memory = pin_memory,
        sampler = valid_sampler
    )

    return train_loader, val_loader

### Transform Functions

In [ ]:
def build_transforms(image_heigt, image_width): 
    
    train_transform = A.Compose([
        A.Resize(height=image_heigt, width=image_width),
        A.Flip(p=0.5),
        A.Normalize(mean=(0.485, 0.456, 0.406), 
                    std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
        ],)

    val_transform = A.Compose([
        A.Resize(height=image_heigt, width=image_width),
        A.Normalize(mean=(0.485, 0.456, 0.406), 
                    std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
        ],)
    
    return train_transform, val_transform

### Accuracy Function 

In [ ]:
def compute_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim=True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

### Evaluation Function

In [ ]:
def evaluate_fn(model, loader, criterion, device):

  epoch_loss = 0
  epoch_acc = 0

  model.eval()
  loop = tqdm(loader, desc="Evaluating", leave=False)

  with torch.no_grad():

    for (x, y) in loop:

      x = x.to(device)
      y = y.to(device)

      y_pred = model(x)

      loss = criterion(y_pred, y)

      acc = compute_accuracy(y_pred, y)

      epoch_loss += loss.item()
      epoch_acc += acc.item()

  return epoch_loss / len(loader), epoch_acc / len(loader)

In [ ]:
def epoch_time(start_time, end_time):
  elapsed_time = end_time - start_time
  elapsed_mins = int(elapsed_time / 60)
  elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
  return elapsed_mins, elapsed_secs

## Configure Sweeps

In [ ]:
# Setup wandb 
!pip install wandb --upgrade
import pprint
import wandb

wandb.login()

In [ ]:
sweep_config = {
    'method': 'random'
    }

metric = {
    'name': 'valiadtion_accuracy',
    'goal': 'maximize'   
    }

sweep_config['metric'] = metric

In [ ]:
parameters_dict = {
    'batch_size': {
        # integers between 2 and 32
        # with evenly-distributed logarithms 
        'distribution': 'q_log_uniform_values',
        'q': 8,
        'min': 2,
        'max': 32,
        },
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1
        },
    'epochs': {'values': [2, 4, 8]
        },
    }

sweep_config['parameters'] = parameters_dict


pprint.pprint(sweep_config)

In [ ]:
# initialize sweep
sweep_id = wandb.sweep(sweep_config, project="ds_project")

## HP Search 

In [ ]:
def train_fn(loader, model, optimizer, criterion, scaler, device):
    
    model.train()
    loop = tqdm(loader, desc="Training", leave=False)
    
    for batch_idx, (data, targets) in enumerate(loop):
        
        data = data.to(device)
        targets = targets.type(torch.LongTensor).to(device)

        # forward
        # with torch.cuda.amp.autocast():
        with torch.autocast(device_type='cuda', dtype=torch.float16):
          predictions = model(data)
          loss = criterion(predictions, targets)

        # backward
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # update tqdm loop
        loop.set_postfix(loss=loss.item())

        torch.cuda.empty_cache()

In [ ]:
IMAGE_HEIGHT = 1000  # 2000 originally
IMAGE_WIDTH = 1000  # 2000 originally

IMGS_DICT = imgs_dict
LABELS_DICT = labels_dict
KEYS = keys

NUM_WORKERS = 2
PIN_MEMORY = True
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:

# main function with wandb
def main(config=None):
    with wandb.init(config=config):
        config = wandb.config
        
        train_transforms, val_transforms = build_transforms(
            image_heigt=IMAGE_HEIGHT,
            image_width=IMAGE_WIDTH
        )
        
        train_loader, validation_loader = get_loaders(
            imgs_dict=IMGS_DICT,
            labels_dict=LABELS_DICT,
            keys=KEYS,
            batch_size=config.batch_size,
            train_transform=train_transforms,
            val_transform=val_transforms,
            num_workers = NUM_WORKERS,
            pin_memory = PIN_MEMORY)
        
        # specify loss function 
        CRITERION = nn.CrossEntropyLoss()
        CRITERION = CRITERION.to(DEVICE)
        
        # initialize model and optimizer
        MODEL = UNET(in_channels=3, out_channels=6).to(device=DEVICE)
        OPTIMIZER = optim.Adam(MODEL.parameters(), lr=config.learning_rate)
        
        SCALER = torch.cuda.amp.GradScaler()
        
        
    for epoch in trange(config.epochs, desc="Epochs"):

        start_time = time.monotonic()
        
        train_fn(loader=train_loader, 
                 model=MODEL, 
                 optimizer=OPTIMIZER, 
                 criterion=CRITERION, 
                 scaler=SCALER,
                 device=DEVICE)
        
        training_loss, training_accuracy = evaluate_fn(model=MODEL, 
                                                       loader=train_loader, 
                                                       criterion=CRITERION, 
                                                       device=DEVICE)
        
        validation_loss, validation_accuracy = evaluate_fn(model=MODEL, 
                                                           loader=validation_loader, 
                                                           criterion=CRITERION, 
                                                           device=DEVICE)
                
        end_time = time.monotonic()

        epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
        print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {training_loss:.3f} | Train Acc: {training_accuracy*100:.2f}%')
        print(f'\t Val. Loss: {validation_loss:.3f} |  Val. Acc: {validation_accuracy*100:.2f}%')
        
        wandb.log({"epoch": epoch, 
                   "validation_accuracy": validation_accuracy,
                   "validation_loss": validation_loss}) 

In [ ]:
wandb.agent(sweep_id, main, count=1)